<a href="https://colab.research.google.com/github/PrzemyslawCh/TTS/blob/main/RS_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Apr  3 11:25:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1. Scraping data from [101soundboards.com](https://www.101soundboards.com/boards/11050-rick-sanchez-sounds-rick-and-morty-season-1)





In [ ]:
# 1.1 installing neccesary libraries

!pip install httpx
import httpx
from bs4 import BeautifulSoup
import re
import os
import json
import asyncio
import requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 8.8 MB/s eta 0:00:00


In [ ]:
# 1.2 Extract link from websites
def extract_links(urls):
    extracted_links = []
    for url in urls:
        response = requests.get(url)
        html = response.text
        board_data_inline = re.search(r'var board_data_inline\s*=\s*({.*});', html).group(1)
        board_data = json.loads(board_data_inline)
        for sound in board_data['sounds']:
            div_id = sound['id']
            transcript = sound['sound_transcript']
            link = f"https://www.101soundboards.com/sounds/{div_id}-{transcript.replace(' ', '-')}"
            extracted_links.append(link)
    return extracted_links

In [ ]:
urls = [
    'https://www.101soundboards.com/boards/11050-rick-sanchez-sounds-rick-and-morty-season-1',
    'https://www.101soundboards.com/boards/11051-rick-sanchez-sounds-rick-and-morty-season-2'
]

links = extract_links(urls)
print(len(links))
print(links)

303
['https://www.101soundboards.com/sounds/59669-A-moment-of-your-time.', 'https://www.101soundboards.com/sounds/59608-After-you-cried-and-crapped-your-pants.', "https://www.101soundboards.com/sounds/59688-And-I-know-that's-not-a-popular-opinion.", 'https://www.101soundboards.com/sounds/59616-Anything-else?', 'https://www.101soundboards.com/sounds/59653-Are-you-people-even-human', 'https://www.101soundboards.com/sounds/59610-Are-you-the-devil-a-demon-leprechaun?', 'https://www.101soundboards.com/sounds/59611-Are-you-the-devil?', 'https://www.101soundboards.com/sounds/59664-Because-I-got-I-got-a-big-you-know-penis-between-my-legs.', "https://www.101soundboards.com/sounds/59657-Because-I'm-having-a-party-bitch.", 'https://www.101soundboards.com/sounds/59598-Bitch.', 'https://www.101soundboards.com/sounds/59594-Block-Papa-nuke.-Nuke,-nuke,-nuke-nuke.', 'https://www.101soundboards.com/sounds/59583-Boy,-you-really-are-17.', 'https://www.101soundboards.com/sounds/59724-Boy,-you-really-got-m

In [ ]:
# 1.3 Download files from websites

import os
import re
import json
import httpx
from bs4 import BeautifulSoup

async def download_mp3s(links):
    async with httpx.AsyncClient() as client:
        # Create the wavs folder if it doesn't exist
        if not os.path.exists("wavs"):
            os.makedirs("wavs")

        # Find the number of existing files in the wavs folder
        existing_files_count = len(os.listdir("wavs"))

        # Download the MP3 files using the extracted links
        for i, url in enumerate(links):
            response = await client.get(url)
            soup = BeautifulSoup(response.text, "html.parser")

            # Check if there's a redirect
            redirect_meta = soup.find("meta", attrs={"http-equiv": "refresh"})
            if redirect_meta:
                content = redirect_meta["content"]
                target_url = re.search("url='(.*?)'", content).group(1)
                response = await client.get(target_url)
                soup = BeautifulSoup(response.text, "html.parser")

            preload_script = soup.find("script", string=re.compile("sound_data_preload"))

            if not preload_script:
                print(f"Failed to find preload_script for URL: {url}")
                print(response.text)
                continue

            preload_text = re.search("sound_data_preload = ({.*?});", preload_script.string).group(1)
            preload_json = json.loads(preload_text)
            download_url = "https://www.101soundboards.com" + preload_json["sound_file_url"]
            file_name = f"{existing_files_count+i+1}.mp3"

            # Save the file in the wavs folder
            file_path = os.path.join("wavs", file_name)

            mp3_data = (await client.get(download_url)).content
            with open(file_path, "wb") as f:
                f.write(mp3_data)
                # print(f"Downloaded {file_path}")

# Pass the extracted links to the download_mp3s function
await download_mp3s(links)

CancelledError: ignored

In [ ]:
# Sort and check i files are in folder

import os

folder_path = "/content/wavs" # replace with your folder path
files = os.listdir(folder_path)

def get_file_number(file_name):
    # Extract the numeric part of the filename and return it as an integer
    return int(''.join(filter(str.isdigit, file_name)))

sorted_files = sorted(files, key=get_file_number)

print(sorted_files)


['1.mp3', '2.mp3', '3.mp3', '4.mp3', '5.mp3', '6.mp3', '7.mp3', '8.mp3', '9.mp3', '10.mp3', '11.mp3', '12.mp3', '13.mp3', '14.mp3', '15.mp3', '16.mp3', '17.mp3', '18.mp3', '19.mp3', '20.mp3', '21.mp3', '22.mp3', '23.mp3', '24.mp3', '25.mp3', '26.mp3', '27.mp3', '28.mp3', '29.mp3', '30.mp3', '31.mp3', '32.mp3', '33.mp3', '34.mp3', '35.mp3', '36.mp3', '37.mp3', '38.mp3', '39.mp3', '40.mp3', '41.mp3', '42.mp3', '43.mp3', '44.mp3', '45.mp3']


In [ ]:
# Create transciption file based on links

import requests
import json
import re

urls = [
    'https://www.101soundboards.com/boards/11050-rick-sanchez-sounds-rick-and-morty-season-1',
    'https://www.101soundboards.com/boards/11051-rick-sanchez-sounds-rick-and-morty-season-2'
]

transcription = []
file_number = 0
for url in urls:
    response = requests.get(url)
    html = response.text
    board_data_inline = re.search(r'var board_data_inline\s*=\s*({.*});', html).group(1)
    board_data = json.loads(board_data_inline)

    for s, sound in enumerate(board_data['sounds']):
        transcript = sound['sound_transcript']
        # Add missing punctuation at the end of the transcript if necessary
        if transcript and transcript[-1] not in ['.', '?', '!']:
            transcript += '.'
        file_number += 1
        filename = f"wavs/{file_number}.wav"
        transcription.append(f"{filename}| {transcript}\n")

# Write transcriptions to file
with open('transcription.txt', 'w') as f:
    f.writelines(transcription)

print("Transcription file has been created successfully!")


Transcription file has been created successfully!


In [ ]:
# Create transciption file based on links

import requests
import json
import re

urls = [
    'https://www.101soundboards.com/boards/11050-rick-sanchez-sounds-rick-and-morty-season-1',
    'https://www.101soundboards.com/boards/11051-rick-sanchez-sounds-rick-and-morty-season-2'
]

transcription = []
file_number = 0
for url in urls:
    response = requests.get(url)
    html = response.text
    board_data_inline = re.search(r'var board_data_inline\s*=\s*({.*});', html).group(1)
    board_data = json.loads(board_data_inline)

    for s, sound in enumerate(board_data['sounds']):
        transcript = sound['sound_transcript']
        file_number += 1
        filename = f"wavs/{file_number}.wav"
        transcription.append(f"{filename}| {transcript}\n")

# Write transcriptions to file
with open('transcription.txt', 'w') as f:
    f.writelines(transcription)

print("Transcription file has been created successfully!")



Transcription file has been created successfully!


## 2. Cleaning data





This supports the Demucs source separation model (https://github.com/facebookresearch/demucs/) This is only for separation with pre-trained models, not training!

You can either upload files manually (slow) or link your Google Drive account.

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

In [ ]:
# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = '/content/wavs'
out_path = '/content/wavs_modified'

In [ ]:
#@title Useful functions, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')

    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()

    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()

    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)


In [ ]:
# Separation to bass, drums, other and vocals

In [ ]:
separate()

In [ ]:
# Moving out vocals to cleaned folder

import os
import shutil

source_dir = "/content/wavs_modified/htdemucs"
dest_dir = "/content/cleaned"

# Create the destination directory if it doesn't exist
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

# Loop through all the folders in the source directory
for folder in os.listdir(source_dir):
    if folder.startswith('.'):
        continue
    # Get the path to the vocals file in the current folder
    vocals_path = os.path.join(source_dir, folder, "vocals.mp3")
    # Create the destination path for the file
    dest_path = os.path.join(dest_dir, folder + ".mp3")
    # Copy the file to the destination directory with the appropriate name
    shutil.copyfile(vocals_path, dest_path)


In [ ]:
# Zipping files

import os
import zipfile

# Set the path to the folder containing the audio files
folder_path = '/content/cleaned'

# Set the path to the folder where you want to save the zipped files
output_path = '/content/Zipped/'

# Create the output folder if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Get a list of all the audio files in the folder
audio_files = [f for f in os.listdir(folder_path) if f.endswith('.mp3') or f.endswith('.wav')]

# Zip each file individually
for file_name in audio_files:
    with zipfile.ZipFile(os.path.join(output_path, file_name + '.zip'), 'w', compression=zipfile.ZIP_DEFLATED) as zip_file:
        zip_file.write(os.path.join(folder_path, file_name), file_name)

print('All audio files have been zipped and saved to', output_path)


All audio files have been zipped and saved to /content/Zipped/


In [ ]:
# Cleaning data in any given folder

import os

directory = '/content/wavs'

for root, dirs, files in os.walk(directory, topdown=False):
    for file in files:
        try:
            os.remove(os.path.join(root, file))
        except OSError as e:
            print(f"Error deleting {file}: {e}")

In [ ]:
# Deleting folders
import os

directory = '/content/wavs'

for root, dirs, files in os.walk(directory, topdown=False):
    for dir in dirs:
        try:
            os.rmdir(os.path.join(root, dir))
        except OSError as e:
            print(f"Error deleting {dir}: {e}")


In [ ]:
# Creating new folder in google drive

!mkdir /content/drive/My\ Drive/RS_Data


In [ ]:
# Moving data to google drive

!cp -r /content/Zipped /content/drive/My\ Drive/RS_Data/
!cp /content/transcription.txt /content/drive/My\ Drive/RS_Data/
